In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from catboost import CatBoostRegressor
import csv

%load_ext autoreload
%matplotlib inline

In [ ]:
# import sklearn
# sklearn.metrics.SCORERS.keys()

## Import data and small preprocessing

In [ ]:
trainData = pd.read_csv("../input/beer-sales/train_preprocessed.csv",dtype={'Store': object,'ts_promo': object,'store_promo': object,'dom_promo': object,'dow_promo': object})
trainData['Date'] = pd.to_datetime(trainData['Date'])
trainData

In [ ]:
testData = pd.read_csv("../input/beer-sales/test_preprocessed.csv",dtype={'Store': object,'ts_promo': object,'store_promo': object,'dom_promo': object,'dow_promo': object})
testData['Date'] = pd.to_datetime(testData['Date'])
testData

In [ ]:
for c in trainData.columns:
    col_type = trainData[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        trainData[c] = trainData[c].astype('category')

In [ ]:
for c in testData.columns:
    col_type = testData[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        testData[c] = testData[c].astype('category')

In [ ]:
print(trainData['Date'].is_monotonic_increasing)
print(testData['Date'].is_monotonic_increasing)
trainData = trainData.sort_values(by=['Date'])
testData = testData.sort_values(by=['Date'])

In [ ]:
X = trainData.drop(['Sales','Date'], axis=1)#Date
y = trainData['Sales']

In [ ]:
var_categ =[]
for c in X.columns:
    col_type = X[c].dtype
    if col_type.name == 'category':
        var_categ.append(c)

# LGBM

## Get best hyperparam

In [ ]:
# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer

# lgbm = LGBMRegressor(
#     silent=True, 
#     n_jobs=-1, 
#     #device = "gpu"
#     )

# search_space = {"boosting_type": Categorical(['dart']),
#         "num_leaves": Integer(6, 60), 
#         "max_depth": Integer(-1, 40), 
#         "learning_rate": Real(0.001,1),
#         "n_estimators": Integer(2, 1000),
#         "subsample_for_bin": Integer(195000, 204000),
#         "colsample_bytree" : Real (0.5,1.0),#New
#         "subsample": Real (0.5,1.0) #New
#     }
# cv = TimeSeriesSplit(n_splits=7)

# bayes_cv_tuner = BayesSearchCV(
#     estimator = lgbm, 
#     search_spaces =search_space, 
#     n_iter=20, # specify how many iterations
#     scoring="neg_root_mean_squared_error",
#     cv=cv,
#     verbose=0,
#     n_jobs =-1
# )

# def print_status(optimal_result):
#     """Shows the best parameters found and accuracy attained of the search so far."""
#     print("Done")

# bayes_cv_tuner.fit(X, y, callback=print_status) # .iloc[1:500]

In [ ]:
# bayes_cv_tuner.best_params_

In [ ]:
# print(type(bayes_cv_tuner.best_params_))

# keys, values = [], []
# for key, value in bayes_cv_tuner.best_params_.items():
#     keys.append(key)
#     values.append(value)       

# with open("LGBMBestParam.csv", "w") as outfile:
#     csvwriter = csv.writer(outfile)
#     csvwriter.writerow(keys)
#     csvwriter.writerow(values)

In [ ]:
# bayes_cv_tuner.best_score_

In [ ]:
# from skopt.plots import plot_convergence

# plot_convergence(result) 

## Apply

In [ ]:
lgbm = LGBMRegressor(
    boosting_type = 'dart',
    learning_rate = 0.35878911553612336, 
    max_depth = 1,
    metric='rmse',
    n_estimators = 524,
    num_leaves = 21,
    subsample_for_bin = 199000,
    colsample_bytree = 0.8,
    subsample = 1.0,
    silent=True
    )
#BestParam
# [('boosting_type', 'dart'),#1
#              ('learning_rate', 0.039741509928932967),
#              ('max_depth', 10), #Max
#              ('n_estimators', 336),
#              ('num_leaves', 26),
#              ('subsample_for_bin', 201000)]) #Max

# [('boosting_type', 'dart'),#2
#              ('learning_rate', 0.4013806578117077),
#              ('max_depth', 14),
#              ('n_estimators', 113),
#              ('num_leaves', 8),
#              ('subsample_for_bin', 199119)])
# ([('boosting_type', 'dart'),#3 best
#              ('colsample_bytree', 0.8),
#              ('learning_rate', 0.35878911553612336),
#              ('max_depth', 1),
#              ('n_estimators', 524),
#              ('num_leaves', 21),
#              ('subsample', 1.0),
#              ('subsample_for_bin', 199000)])
# [('boosting_type', 'dart'), #4
#              ('colsample_bytree', 0.7423119611889375),
#              ('learning_rate', 0.7376761996696818),
#              ('max_depth', 1),
#              ('n_estimators', 61),
#              ('num_leaves', 7),
#              ('subsample', 0.6455963683564654),
#              ('subsample_for_bin', 204000)])

In [ ]:
#TSCV
# tscv = TimeSeriesSplit(n_splits=7)
# rmse=[]
# for train_index, test_index in tscv.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     lgbm.fit(X_train,y_train)
#     lgbm_prediction = lgbm.predict(X_test)
#     rmse.append(round(mean_squared_error(lgbm_prediction, y_test,squared=False), 2))
    
# average_rmse = np.mean(rmse)
# print(average_rmse)

testX = testData.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
lgbm.fit(X,y)
lgbm_prediction = lgbm.predict(testX)

testData['Sales'] = lgbm_prediction
testData = testData.sort_values(by=['id'])
print(testData)
testData[['id','Sales']].to_csv("LGBMsubmission.csv", index=False)


In [ ]:
importances = lgbm.feature_importances_
forest_importances = pd.Series(importances, index=X.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(ax=ax)
ax.set_title("Gini Importance")
ax.set_ylabel("Importance")
fig.tight_layout()

# CatBoost

## Get best hyperparam

In [ ]:
# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer
        
# catboost = CatBoostRegressor(
#     silent=True, 
#     #task_type = "GPU",
#     cat_features=var_categ
#     )

# search_space = {
#         'iterations': Integer(10, 800),
#     #CatBoostError: only one of the parameters iterations, n_estimators, num_boost_round, num_trees should be initialized.

#         #"n_estimators": Integer(6, 500), 
#         "learning_rate": Real(0.01,1),
#         'random_strength': Real(1e-9, 10, 'log-uniform'),
#         "depth": Integer(1, 10),
#         "bagging_temperature": Real(0.0, 1.0),
#         'border_count': Integer(1, 255),
#         'l2_leaf_reg': Integer(2, 30),               
#     }


# cv = TimeSeriesSplit(n_splits=7)

# bayes_cv_tuner = BayesSearchCV(
#     estimator = catboost, 
#     search_spaces =search_space, 
#     n_iter= 15, # specify how many iterations
#     scoring="neg_root_mean_squared_error",
#     cv=cv,
#     verbose=0,
#     n_jobs =-1
# )

# def print_status(optimal_result):
#     """Shows the best parameters found and accuracy attained of the search so far."""
#     print("DONE")
# #     models_tested = pd.DataFrame(bayes_cv_tuner.cv_results_)
# #     best_parameters_so_far = pd.Series(bayes_cv_tuner.best_params_)
# #     print(
# #         "Model #{}\nBest RMSE so far: {}\nBest parameters so far: {}\n".format(
# #             len(models_tested),
# #             np.round(bayes_cv_tuner.best_score_, 3),
# #             bayes_cv_tuner.best_params_,
# #         )
# #     )

# #     clf_type = bayes_cv_tuner.estimator.__class__.__name__
# #     models_tested.to_csv(clf_type + "_cv_results_summary.csv")

# bayes_cv_tuner.fit(X, y, callback=print_status)# .iloc[1:500]

In [ ]:
# bayes_cv_tuner.best_params_

In [ ]:
# print(type(bayes_cv_tuner.best_params_))

# keys, values = [], []
# for key, value in bayes_cv_tuner.best_params_.items():
#     keys.append(key)
#     values.append(value)       

# with open("CatboostBestParam.csv", "w") as outfile:
#     csvwriter = csv.writer(outfile)
#     csvwriter.writerow(keys)
#     csvwriter.writerow(values)

In [ ]:
# bayes_cv_tuner.best_score_

## Apply

In [ ]:
catboost = CatBoostRegressor(
    n_estimators=64,
    learning_rate=0.14038578250228806,
    depth=8,
    bagging_temperature=0.1075748346436656,
    border_count = 213,
    l2_leaf_reg = 12,
    random_strength = 5.03421359307061e-09,
    cat_features=var_categ,
    thread_count=-1,
    silent=True
)
#BestParam
# [('bagging_temperature', 0.42802269671472565), #1
#              ('border_count', 180),
#              ('depth', 2),
#              ('l2_leaf_reg', 13),
#              ('learning_rate', 0.09424480742594872),
#              ('n_estimators', 60),
#              ('random_strength', 9.029905504478138e-08)]

# ([('bagging_temperature', 0.9094652241987762), #2
#              ('border_count', 223),
#              ('depth', 1),
#              ('l2_leaf_reg', 4),
#              ('learning_rate', 0.01058603000501733),
#              ('n_estimators', 82),
#              ('random_strength', 1.272165412113495e-08)])

# [('bagging_temperature', 0.1075748346436656), #3 best
#              ('border_count', 213),
#              ('depth', 8),
#              ('l2_leaf_reg', 12),
#              ('learning_rate', 0.14038578250228806),
#              ('n_estimators', 64),
#              ('random_strength', 5.03421359307061e-09)])

# [('bagging_temperature', 1.0), #4
#              ('border_count', 112),
#              ('depth', 8),
#              ('l2_leaf_reg', 30),
#              ('learning_rate', 0.38487817448416656),
#              ('n_estimators', 6),
#              ('random_strength', 0.0004726359309565526)])
# [('bagging_temperature', 0.37166939185644426), #5
#              ('border_count', 255),
#              ('depth', 1),
#              ('iterations', 285),
#              ('l2_leaf_reg', 19),
#              ('learning_rate', 0.01),
#              ('random_strength', 10.0)])

In [ ]:
#TSCV
# tscv = TimeSeriesSplit(n_splits=7)#4 months each
# rmse=[]
# for train_index, test_index in tscv.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#     catboost.fit(X_train,y_train)
#     catboost_prediction = catboost.predict(X_test)
#     rmse.append(round(mean_squared_error(catboost_prediction, y_test,squared=False), 2))
    
# average_rmse = np.mean(rmse)
# print(average_rmse)

testX = testData.drop(['id','Date'], axis=1)
if 'Sales' in testX.columns:
    testX = testX.drop(['Sales'], axis=1)

#Now all data
catboost.fit(X,y)
catboost_prediction = catboost.predict(testX)
print(catboost_prediction)
testData['Sales'] = catboost_prediction
testData = testData.sort_values(by=['id'])
print(testData)
testData[['id','Sales']].to_csv("catboostSubmission.csv", index=False)


In [ ]:
importances = catboost.feature_importances_
forest_importances = pd.Series(importances, index=X.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar(ax=ax)
ax.set_title("Gini Importance")
ax.set_ylabel("Importance")
fig.tight_layout()

In [ ]:
# Stacking different algorithm, different input (only the prediction, Test+Sales), different hyperparam
#LGBM,Linear Reg, XGBoost
#For each try with only prediction input and different hyperapram, with test+sales and automize bestg hyperparam